In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, ConcatDataset
from torchvision import datasets, transforms, models
from torchvision.models import efficientnet_b3, EfficientNet_B3_Weights
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from tqdm import tqdm

ModuleNotFoundError: No module named 'tqdm'

In [2]:
data_dir = "/kaggle/input/brain-tumor-mri-dataset/"
train_dir = os.path.join(data_dir, "Training")
test_dir = os.path.join(data_dir, "Testing")

def make_final_transforms(aug_level="flip_rot", rot_deg=12):
    if aug_level == "flip_rot":
        tf_train = transforms.Compose([
            transforms.Resize((300, 300)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(rot_deg),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406],
                                 [0.229, 0.224, 0.225])
        ])
    else:
        tf_train = transforms.Compose([
            transforms.Resize((300, 300)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406],
                                 [0.229, 0.224, 0.225])
        ])
    tf_test = transforms.Compose([
        transforms.Resize((300, 300)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
    return tf_train, tf_test

In [3]:
tf_train, tf_test = make_final_transforms("flip_rot", 12)
train_data = datasets.ImageFolder(train_dir, transform=tf_train)
test_data = datasets.ImageFolder(test_dir, transform=tf_test)

val_ratio = 0.15
train_len = int((1 - val_ratio) * len(train_data))
val_len = len(train_data) - train_len
train_subset, val_subset = random_split(train_data, [train_len, val_len], generator=torch.Generator().manual_seed(42))

final_train = ConcatDataset([train_subset, val_subset])
train_loader = DataLoader(final_train, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

In [4]:
def build_final_model(dropout=0.4626, freeze_ratio=0.449):
    model = efficientnet_b3(weights=EfficientNet_B3_Weights.DEFAULT)
    n_layers = int(len(list(model.features)) * freeze_ratio)
    for i, layer in enumerate(model.features):
        for param in layer.parameters():
            param.requires_grad = i >= n_layers

    model.classifier[1] = nn.Sequential(
        nn.Dropout(dropout),
        nn.Linear(model.classifier[1].in_features, 4)
    )
    return model.to(device)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
def run_epoch(model, loader, optimizer, criterion, device, train=True):
    model.train() if train else model.eval()
    loss_total, correct, total = 0.0, 0, 0
    with torch.set_grad_enabled(train):
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            if train: optimizer.zero_grad()
            out = model(x)
            loss = criterion(out, y)
            if train: loss.backward(); optimizer.step()
            loss_total += loss.item() * y.size(0)
            correct += (out.argmax(1) == y).sum().item()
            total += y.size(0)
    return loss_total / total, 100. * correct / total

In [9]:
model = build_final_model(dropout=0.4626, freeze_ratio=0.449)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=9.83e-5, weight_decay=1.55e-6)

for epoch in tqdm(range(15)):
    train_loss, train_acc = run_epoch(model, train_loader, optimizer, criterion, device, train=True)
    print(f"Epoch {epoch+1}: Train Loss = {train_loss:.4f}, Train Acc = {train_acc:.2f}%")

  7%|▋         | 1/15 [01:12<16:57, 72.69s/it]

Epoch 1: Train Loss = 0.6604, Train Acc = 78.22%


 13%|█▎        | 2/15 [02:25<15:47, 72.85s/it]

Epoch 2: Train Loss = 0.1712, Train Acc = 94.12%


 20%|██        | 3/15 [03:39<14:37, 73.09s/it]

...

Epoch 13: Train Loss = 0.0088, Train Acc = 99.70%


 93%|█████████▎| 14/15 [17:05<01:13, 73.27s/it]

Epoch 14: Train Loss = 0.0067, Train Acc = 99.75%


100%|██████████| 15/15 [18:18<00:00, 73.26s/it]

Epoch 15: Train Loss = 0.0090, Train Acc = 99.72%


In [10]:
test_loss, test_acc = run_epoch(model, test_loader, optimizer, criterion, device, train=False)
print(f"\n🎯 Final Test Accuracy: {test_acc:.2f}%")


🎯 Final Test Accuracy: 99.54%


In [12]:
torch.save(model.state_dict(), "tumor_model.pth")